# ANM (Anisotropic Network Model)

## Directional springs along interatomic lines

For each connected pair $(i,j)$:
- Relative vector: $\mathbf{d}_{ij}=\mathbf{R}_j-\mathbf{R}_i$, distance $d_{ij}=\|\mathbf{d}_{ij}\|$, unit $\hat{\mathbf{r}}_{ij}=\mathbf{d}_{ij}/d_{ij}$.
- Spring resists stretching along $\hat{\mathbf{r}}_{ij}$ (no transverse resistance).
- Stiffness $k_{ij}\ge 0$.

Off-diagonal $3\times3$ block for $i\ne j$:
$$
H_{ij} = -\,k_{ij}\, \hat{\mathbf{r}}_{ij}\hat{\mathbf{r}}_{ij}^{\top}.
$$

Diagonal blocks enforce force balance:
$$
H_{ii} = -\sum_{j\ne i} H_{ij}.
$$

Properties:
- $H$ is symmetric and positive semidefinite.
- Translations and rotations are null; later.


In [2]:
from typing import Callable, List, Tuple
import numpy as np

def build_anm_hessian(
    R: np.ndarray,
    edges: List[Tuple[int, int]],
    kmap: Callable[[int, int], float] | float = 1.0,
) -> np.ndarray:
    """Build 3N x 3N ANM stiffness matrix H."""
    R = np.asarray(R, float)
    N = R.shape[0]
    H = np.zeros((3*N, 3*N), float)
    if callable(kmap):
        kfun = kmap
    else:
        const_k = float(kmap)
        kfun = lambda i, j: const_k
    for (i, j) in edges:
        rij = R[j] - R[i]
        d = float(np.linalg.norm(rij))
        if d == 0.0:
            continue
        u = rij / d
        kij = float(kfun(i, j))
        K = -kij * np.outer(u, u)
        H[3*i:3*i+3, 3*j:3*j+3] += K
        H[3*j:3*j+3, 3*i:3*i+3] += K.T
        H[3*i:3*i+3, 3*i:3*i+3] -= K
        H[3*j:3*j+3, 3*j:3*j+3] -= K
    H = 0.5 * (H + H.T)
    return H


## Testing

- Geometry and edges (O–H bonds),
- Build $H$,
- Check symmetry and near-null rigid motions,
- Inspect eigenvalues (expect ~6 near 0, 3 positive).


In [4]:
%run io.ipynb
%run geometry.ipynb
%run invariance_test.ipynb

Name:  H2O
N atoms: 3
atoms: ['O', 'H', 'H']
coords (Å):
 [[ 0.        0.        0.119262]
 [ 0.        0.763239 -0.477047]
 [ 0.       -0.763239 -0.477047]]
masses (amu): [15.999  1.008  1.008]
edges: [(0, 1), (0, 2), (1, 2)]
Name:  H2O
Original COM:
 [0.       0.       0.052531]
Recomputed COM from Rc:
 [0.00000000e+00 0.00000000e+00 1.23255401e-17]
Name:  H2O
Original COM:
 [0.       0.       0.052531]
Recomputed COM from Rc:
 [0.00000000e+00 0.00000000e+00 1.23255401e-17]
Name:  H2O
original atom:
 [[ 0.        0.        0.119262]
 [ 0.        0.763239 -0.477047]
 [ 0.       -0.763239 -0.477047]]
atom before random rigid move:
 [[ 0.        0.        0.066731]
 [ 0.        0.763239 -0.529578]
 [ 0.       -0.763239 -0.529578]]
Random rotation matrix:
 [[-0.48146442 -0.47052726  0.73945663]
 [ 0.87638925 -0.2695883   0.39907898]
 [ 0.01157132  0.84019417  0.5421622 ]]
Random translation vector:
 [-0.5748316  -0.42566182 -0.08281721]
atom after random rigid move:
 [[-0.52548692 -0.399

For a uniform translation $\Delta\mathbf R_i=\mathbf a$ for all $i$,
$$
\Delta\mathbf R_j-\Delta\mathbf R_i=\mathbf 0
;\Rightarrow;
\hat{\mathbf u}_{ij}\cdot(\Delta\mathbf R_j-\Delta\mathbf R_i)=0
;\Rightarrow;
E=0.
$$

Therefore each translational direction $\mathbf t^{(x)},\mathbf t^{(y)},\mathbf t^{(z)}$ satisfies
$$
H,\mathbf t^{(a)}=\mathbf 0,\qquad a\in{x,y,z},
$$
so the matrix of translations $T=[\mathbf t^{(x)}\ \mathbf t^{(y)}\ \mathbf t^{(z)}]$ obeys
$$
|H,T|_\infty \approx 0.
$$


Use COM-centred coordinates $\mathbf r_i=\mathbf R_i-\mathbf R_{\mathrm{COM}}$. A small rigid rotation with angular vector $\boldsymbol\Omega$ gives
$$
\Delta\mathbf R_i=\boldsymbol\Omega\times \mathbf r_i.
$$
For an edge $(i,j)$ with chord $\mathbf d_{ij}=\mathbf r_j-\mathbf r_i$ (parallel to $\hat{\mathbf u}{ij}$),
$$
\Delta\mathbf R_j-\Delta\mathbf R_i=\boldsymbol\Omega\times \mathbf d{ij},
$$
and the longitudinal projection entering the energy is
$$
\hat{\mathbf u}{ij}\cdot(\boldsymbol\Omega\times \mathbf d{ij})
= (\hat{\mathbf u}{ij}\times \boldsymbol\Omega)\cdot \mathbf d{ij}
= (\hat{\mathbf u}{ij}\times \boldsymbol\Omega)\cdot \big(\lVert \mathbf d{ij}\rVert \hat{\mathbf u}{ij}\big)
= 0,
$$
since $\hat{\mathbf u}{ij}\times \boldsymbol\Omega \perp \hat{\mathbf u}{ij}$. Hence $E=0$ for any rigid rotation, and with $\mathrm{Rot}\in\mathbb R^{3N\times 3}$ collecting the three rotation directions,
$$
|H,\mathrm{Rot}|\infty \approx 0.
$$


In [ ]:
import numpy.linalg as LA
import numpy as np
from ase.build import molecule

atoms = molecule('H2O')
symbols = atoms.get_chemical_symbols()
R = atoms.get_positions()       # (N,3) Å
masses = atoms.get_masses()          # (N,) amu

edges = guess_bonds(symbols, R, scale=1.2) # from io.ipynb

H = build_anm_hessian(R, edges, kmap=1.0)

sym_err = LA.norm(H - H.T, ord=np.inf)

Rc, _ = shift_to_com(R, masses) # from geometry.ipynb
T = rigid_translation_vectors(N=R.shape[0]) # from geometry.ipynb
Rot = rigid_rotation_vectors(Rc) # from geometry.ipynb

res_T = LA.norm(H @ T, ord=np.inf)
res_R = LA.norm(H @ Rot, ord=np.inf)

print("Symmetry error ||H-H^T||_inf:", sym_err)
print("||H @ translations||:", res_T)
print("||H @ rotations||:", res_R)


Symmetry error ||H-H^T||_inf: 0.0
||H @ translations||: 0.0
||H @ rotations||: 1.2373361711366492e-16


In [ ]:
w = np.sort(LA.eigvalsh(H))
print("Eigenvalues:", w)

Eigenvalues: [-1.20842525e-16 -7.94498982e-17 -1.24802339e-17  0.00000000e+00
  1.26659237e-33  4.06674948e-17  1.22355621e-16  1.75808022e+00
  2.24191978e+00]
